In [7]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
from pymongo import MongoClient
import db_con_mongo

OperationFailure: Authentication failed.

DATA SOURCE : https://www.cs.york.ac.uk/semeval-2013/task9/index.php%3Fid=data.html

In [ ]:
file_locs = ['/home/grace/workspace/bio_dataset/semeval_task9_train_pair/Train/MedLine',
            '/home/grace/workspace/bio_dataset/semeval_task9_train_pair/Train/DrugBank']

entity_pairs = []
sents = []
for file_loc in file_locs:
    medlines = os.listdir(file_loc)

    for med in medlines:
        doc = minidom.parse(file_loc+'/'+med)
        items = doc.getElementsByTagName('document')
        doc_id = items[0].attributes['id'].value

        for sent in doc.getElementsByTagName('sentence'):
            sent_id = sent.attributes['id'].value
            sent_text = sent.attributes['text'].value
            if sent_text.strip():
                sents.append({'id':sent_id, 'sent':sent_text})

            # entity(drug)의 갯수가 1보다 클때만
            if len(sent.getElementsByTagName('pair')) > 0:
                for pair_info in sent.getElementsByTagName('pair'):
                    ddi = pair_info.attributes['ddi'].value
                    e1 = pair_info.attributes['e1'].value
                    e2 = pair_info.attributes['e2'].value
                    
                    for e in sent.getElementsByTagName('entity'):
                        if e.attributes['id'].value==e1:
                            e1_name = e.attributes['text'].value
                        elif e.attributes['id'].value==e2:
                            e2_name = e.attributes['text'].value
                    
                    if e1_name < e2_name:
                        entity_pairs.append({'ddi':ddi, 
                                             'e1_id':e1, 
                                             'e1_name':e1_name,
                                             'e2_id':e2, 
                                             'e2_name':e2_name})
                    else:
                        entity_pairs.append({'ddi':ddi, 
                                             'e1_id':e2, 
                                             'e1_name':e2_name,
                                             'e2_id':e1, 
                                             'e2_name':e1_name})
    #                 for ent in sent.getElementsByTagName('entity'):
    #                     ent_id = ent.attributes['id'].value
    #         #             ent_charOffset = ent.attributes['charOffset'].value
    #                     ent_type = ent.attributes['type'].value.strip()
    #                     ent_text = ent.attributes['text'].value.lower().strip()

    #                     entities.append({'type':ent_type, 'entity':ent_text})
    #                 entity_pairs.append(entities)

In [ ]:
ents = pd.DataFrame(entity_pairs)
print(ents.shape)

ents.drop_duplicates(inplace=True)
print(ents.shape)
# ents.to_csv('./entity_dic_ddi.csv', sep=',', header=None, index=None)

In [ ]:
ents.head()

In [ ]:
ents.ddi = np.where(ents.ddi=='true', True, False)

In [ ]:
ents.ddi.unique()

In [ ]:
# ents.loc[ents.e2_id.str.find('DDI-DrugBank.d493.s0.e0')>=0]

In [ ]:
ents.loc[ents.ddi==False].shape

In [ ]:
ents.loc[ents.ddi==True].shape

In [ ]:
duplicated_ents = ents[ents.duplicated(subset=['e1_name', 'e2_name'])]
print(duplicated_ents)

In [ ]:
ents['True_Pair_Exists'] = False
ents.loc[ents.ddi==True, 'True_Pair_Exists'] = True

In [ ]:
# ents.loc[ents.ddi==True]
# ents.loc[ents.ddi==False]

In [ ]:
# id로는 식별 불가
ents.loc[(ents.e1_id=='DDI-MedLine.d8.s6.e1') | (ents.e2_id=='DDI-MedLine.d8.s6.e1')]

In [ ]:
for ind, en in ents.loc[ents.ddi==True].iterrows():
#     print(en)
    ents.loc[(ents.e1_name==en[2]) & (ents.e2_name==en[4]) & (ents.ddi==False), 'True_Pair_Exists'] = True
    ents.loc[(ents.e1_name==en[4]) & (ents.e2_name==en[2]) & (ents.ddi==False), 'True_Pair_Exists'] = True

In [ ]:
ents.head()

In [ ]:
ents.loc[ents.True_Pair_Exists==False].shape

In [ ]:
## MongoDB에 들어있는 DrugBank Data 확인하기

In [ ]:
client = db_con_mongo.get_connection()
print(client)

In [ ]:
print(client.ddi)
ddi = client.ddi

In [ ]:
drugbank = ddi.get_collection('drugbank')

In [ ]:
# cursor = drugbank.find({'structured_drug_interactions': {'$size': {'$gt':1}}})
cursor = drugbank.find({'drugbank_id': 'DB00001'})
print(len(list(cursor)))

In [ ]:
pd.DataFrame(sents).to_csv('./ddi_sent.csv', sep=',', index=None)
pd.DataFrame(sents).to_csv('./ddi_sent_wo_header.csv', sep=',', header=None, index=None)

In [ ]:
pd.read_csv('./ddi_sent.csv', sep=',').head()

In [ ]:
df_sent = pd.DataFrame(sents)
df_sent.loc[df_sent.sent.isnull()]

In [ ]:
df_sent.shape